In [1]:
import random
from datetime import datetime
import numpy as np
import pandas as pd

import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

from faster_whisper import WhisperModel
from tensorflow.keras.models import load_model
from transformers import pipeline

2024-05-06 10:03:03.203973: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-06 10:03:05.249412: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-06 10:03:08.126069: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Loading Text to speech models

In [2]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"

# model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler_tts_mini_v0.1").to(device)
# tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler_tts_mini_v0.1")

# Loading the Emotion database

In [3]:
pre_df = pd.read_pickle('emotion_time_df.csv')
X_test = pre_df['emotion'][:1]

# Loading the LSTM Module for prediction

In [4]:
lstm_model = load_model("training_1/cp.keras")
train_predictions = lstm_model.predict(X_test).flatten()

2024-05-06 10:03:14.421186: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-06 10:03:14.421843: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step


# Sentiment Analysis Module

In [5]:
class Sentiment_Analysis:
    def __init__(self):
        pass

    def load_sentiment_model(self, task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None):
        # self.tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
        # self.model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
        self.classifier = pipeline(task=task, model=model, top_k=top_k)
        return self.classifier

In [6]:
def call_sentiment(prompt, sentiment_model):
        sentences = [prompt]
        
        model_outputs = sentiment_model(sentences)
        return model_outputs[0]

In [7]:
train_predictions

array([0.24078234], dtype=float32)

# Game Function

In [8]:
import tkinter as tk
from tkinter import messagebox
import time

def play_game():
    class MemoryGame:
        def __init__(self, master):
            self.master = master
            self.master.title("Memory Boosting Game")

            self.sequence = []
            self.player_sequence = []
            self.num_squares = 3
            self.turn = 0

            self.create_widgets()
            self.generate_sequence()

        def create_widgets(self):
            self.label = tk.Label(self.master, text="Click on the squares in the same order as shown.")
            self.label.pack()

            self.button_frame = tk.Frame(self.master)
            self.button_frame.pack()

            self.buttons = []
            for i in range(self.num_squares):
                button = tk.Button(self.button_frame, width=4, height=2, command=lambda idx=i: self.click_square(idx))
                button.grid(row=0, column=i, padx=5, pady=5)
                self.buttons.append(button)

            self.new_game_button = tk.Button(self.master, text="New Game", command=self.new_game)
            self.new_game_button.pack()

        def generate_sequence(self):
            self.sequence = [random.randint(0, self.num_squares - 1) for _ in range(self.num_squares)]
            self.show_sequence()

        def show_sequence(self):
            for idx in self.sequence:
                self.master.after(1000, self.highlight_square, idx)
                time.sleep(0.5)
                self.master.update()
                self.master.after(1000, self.unhighlight_square, idx)
                time.sleep(0.5)
                self.master.update()

        def highlight_square(self, idx):
            self.buttons[idx].config(bg="blue")

        def unhighlight_square(self, idx):
            self.buttons[idx].config(bg="SystemButtonFace")
            self.turn += 1
            if self.turn == len(self.sequence):
                self.turn = 0

        def click_square(self, idx):
            self.player_sequence.append(idx)
            if self.player_sequence == self.sequence:
                messagebox.showinfo("Congratulations!", "You won!")
                self.ask_continue()

            elif self.player_sequence != self.sequence[:len(self.player_sequence)]:
                messagebox.showerror("Wrong Sequence", "Wrong sequence! Try again.")
                self.player_sequence = []

        def ask_continue(self):
            answer = messagebox.askquestion("Continue?", "Do you want to continue playing?")
            if answer == "yes":
                self.new_game()
            else:
                self.master.destroy()

        def new_game(self):
            self.sequence = []
            self.player_sequence = []
            self.turn = 0
            self.generate_sequence()

    root = tk.Tk()
    game = MemoryGame(root)
    root.mainloop()

# Condition for the patient to play a game

In [9]:
emotions = {
    'joy': 0.0,
    'excitement': 0.0,
    'optimism': 0.0,
    'love': 0.0,
    'amusement': 0.0,
    'gratitude': 0.0,
    'surprise': 0.0,
    'relief': 0.0,
    'pride': 0.0,
    'neutral': 0.0,
    'approval': 0.0,
    'realization': 0.0,
    'admiration': 0.0,
    'caring': 0.0,
    'curiosity': 0.0,
    'embarrassment': 0.0,
    'nervousness': 0.0,
    'sadness': 0.0,
    'disappointment': 0.0,
    'confusion': 0.0,
    'disapproval': 0.0,
    'fear': 0.0,
    'desire': 0.0,
    'grief': 0.0,
    'remorse': 0.0,
    'annoyance': 0.0,
    'anger': 0.0,
    'disgust': 0.0}

In [10]:
emotion_ids = {
    'joy': 0,
    'excitement': 1,
    'optimism': 2,
    'love': 3,
    'amusement': 4,
    'gratitude': 5,
    'surprise': 6,
    'relief': 7,
    'pride': 8,
    'neutral': 9,
    'approval': 10,
    'realization': 11,
    'admiration': 12,
    'caring': 13,
    'curiosity': 14,
    'embarrassment': 15,
    'nervousness': 16,
    'sadness': 17,
    'disappointment': 18,
    'confusion': 19,
    'disapproval': 20,
    'fear': 21,
    'desire': 22,
    'grief': 23,
    'remorse': 24,
    'annoyance': 25,
    'anger': 26,
    'disgust': 27
}

In [11]:
if train_predictions  < 19:
    
    # prompt = "Hey would you want to play a game?"
    # description = "A mature male voice with a slight British accent, speaking in a professional hospital setting."
    
    # input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
    # prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    
    # generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
    # audio_arr = generation.cpu().numpy().squeeze()
    # sf.write("parler_tts_out.wav", audio_arr, model.config.sampling_rate)

    # input("Enter you choice")
    #play audio
    model_size = "medium"  # medium is better 
    model = WhisperModel(model_size, device="cpu", compute_type="float32")
    print("Do you want to play a game?")

    audio_path="parler_tts_out.mp3"
    beam_size = 20  # You can adjust the beam size as needed
    segments, info = model.transcribe(audio_path, beam_size=beam_size)
    
    del model

    prompt = ""
    for segment in segments:
        prompt += segment.text
    # print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
    print(prompt)

    sentiment_obj = Sentiment_Analysis()
    sentiment_model = sentiment_obj.load_sentiment_model(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None)

    sentiment_list = call_sentiment(prompt, sentiment_model)
    print(sentiment_list)
    max_ = 0.0
    for senti in sentiment_list:
        if senti['score'] > max_:
            max_ = senti['score']
            max_emotion = emotion_ids[senti['label']]
            
    print(max_emotion)
    if max_emotion <= 9:
        play_game()
        print("Game")

        max_ = 0.0

        for senti in sentiment_list:
            emotions[senti['label']] = round(senti['score'], 4)
        
            if emotions[senti['label']] > max_:
                max_ = emotions[senti['label']]
                max_emotion = emotion_ids[senti['label']]
    
        normal_list = []
        for i in emotions:
            normal_list.append(emotions[i])
            
        now = datetime.now()
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
        

        df_list = []

        df_list.append(dt_string)
        for i in normal_list:
            df_list.append(i)
        df_list.append(max_emotion)
        
        outcome = 1.0
        df_list.append(outcome)
        pre_df = pd.read_pickle('emotion_time_df.csv')

        print("Length of df before appending :",len(pre_df))
        pre_df.loc[len(pre_df)] = df_list
        
        print("Length of df after appending :",len(pre_df))
        pre_df.to_pickle('emotion_time_df.csv')
        
    else:
        print("Sentiment is not favorable for playing the game.")
        max_ = 0.0

        for senti in sentiment_list:
            emotions[senti['label']] = round(senti['score'], 4)
        
            if emotions[senti['label']] > max_:
                max_ = emotions[senti['label']]
                max_emotion = emotion_ids[senti['label']]
    
        normal_list = []
        for i in emotions:
            normal_list.append(emotions[i])
            
        now = datetime.now()
        dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
        

        df_list = []

        df_list.append(dt_string)
        for i in normal_list:
            df_list.append(i)
        df_list.append(max_emotion)
        
        outcome = 0.0
        df_list.append(outcome)
        pre_df = pd.read_pickle('emotion_time_df.csv')

        print("Length of df before appending :",len(pre_df))
        pre_df.loc[len(pre_df)] = df_list
        
        print("Length of df after appending :",len(pre_df))
        pre_df.to_pickle('emotion_time_df.csv')
    
else:
    print("Continuting to monitor")

Do you want to play a game?
 The family members worked together to achieve their goals, whether it was participating in a talent show completing a fitness challenge, gardening competition, family reunion, camping trip, home renovation project, surprise birthday party, community clean-up day, fishing expedition, or family game night.


/root/anaconda3/envs/final_year/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[{'label': 'neutral', 'score': 0.5647067427635193}, {'label': 'excitement', 'score': 0.0775851309299469}, {'label': 'approval', 'score': 0.0721072182059288}, {'label': 'optimism', 'score': 0.03795459121465683}, {'label': 'desire', 'score': 0.03590855374932289}, {'label': 'joy', 'score': 0.02731928415596485}, {'label': 'admiration', 'score': 0.026642562821507454}, {'label': 'realization', 'score': 0.018112912774086}, {'label': 'surprise', 'score': 0.00815604068338871}, {'label': 'gratitude', 'score': 0.005090021528303623}, {'label': 'pride', 'score': 0.004654537886381149}, {'label': 'relief', 'score': 0.003932066727429628}, {'label': 'love', 'score': 0.0031295192893594503}, {'label': 'caring', 'score': 0.002919531427323818}, {'label': 'disappointment', 'score': 0.0026826756075024605}, {'label': 'curiosity', 'score': 0.0019496805034577847}, {'label': 'annoyance', 'score': 0.001654338208027184}, {'label': 'sadness', 'score': 0.0013942057266831398}, {'label': 'amusement', 'score': 0.001202

Exception in Tkinter callback
Traceback (most recent call last):
  File "/root/anaconda3/envs/final_year/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/root/anaconda3/envs/final_year/lib/python3.10/tkinter/__init__.py", line 839, in callit
    func(*args)
  File "/tmp/ipykernel_244820/3471205431.py", line 52, in unhighlight_square
    self.buttons[idx].config(bg="SystemButtonFace")
  File "/root/anaconda3/envs/final_year/lib/python3.10/tkinter/__init__.py", line 1675, in configure
    return self._configure('configure', cnf, kw)
  File "/root/anaconda3/envs/final_year/lib/python3.10/tkinter/__init__.py", line 1665, in _configure
    self.tk.call(_flatten((self._w, cmd)) + self._options(cnf))
_tkinter.TclError: unknown color name "SystemButtonFace"
Exception in Tkinter callback
Traceback (most recent call last):
  File "/root/anaconda3/envs/final_year/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)

Game
Length of df before appending : 70001
Length of df after appending : 70002


In [ ]:
# model = load_model('model/')
# train_predictions = model.predict(X_train).flatten()

In [ ]:
# sentiment_threshold = float(0.5)
# random_row = df.sample(10)
# sentiment_value = random_row['emotion'].values[0]
# print('sentiment value is ',sentiment_value)
# if float(sentiment_value) < sentiment_threshold:
#     # play_game()
#     print("Game")
# else:
#     print("Sentiment is not favorable for playing the game.")
#     # print(type(sentiment_value))